In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\rautl\\OneDrive\\Desktop\\Generative-ai\\End-to-end-medical-chatbot-generative-ai\\research'

In [33]:
import os
os.chdir("../")

In [31]:
%pwd

'c:\\Users\\rautl\\OneDrive\\Desktop\\Generative-ai\\End-to-end-medical-chatbot-generative-ai\\research'

In [35]:
import os

target = "End-to-end-medical-chatbot-generative-ai-main"
alt = "End-to-end-medical-chatbot-generative-ai"

# Try the primary target first, then a common alternative.
if os.path.isdir(target):
	os.chdir(target)
	print("Changed directory to", os.path.abspath(target))
elif os.path.isdir(alt):
	os.chdir(alt)
	print("Primary directory not found — changed directory to alternative:", os.path.abspath(alt))
else:
	# If neither exists, print helpful diagnostics and continue without raising an error.
	print(f"Neither '{target}' nor '{alt}' found in {os.getcwd()}.")
	print("Directory contents:", os.listdir("."))
	print("Continuing in the current working directory.")



Primary directory not found — changed directory to alternative: c:\Users\rautl\OneDrive\Desktop\Generative-ai\End-to-end-medical-chatbot-generative-ai\End-to-end-medical-chatbot-generative-ai


In [36]:
%pwd

'c:\\Users\\rautl\\OneDrive\\Desktop\\Generative-ai\\End-to-end-medical-chatbot-generative-ai'

In [37]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [39]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [49]:
extracted_data = load_pdf_file(data="Data")



In [51]:
# Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [52]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 6973


In [55]:
# text_chunks

In [56]:
# Create Hugging Face Embeddings 


from langchain_community.embeddings import HuggingFaceEmbeddings





In [57]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [58]:
embeddings = download_hugging_face_embeddings()

In [65]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
# query_result

In [60]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [61]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [67]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

pc.create_index(
    name="medicalbot",
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

print(pc.list_indexes())


[{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-kd1um55.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}]


In [68]:
import os

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

os.environ["GROQ_API_KEY"] = GROQ_API_KEY



In [69]:
from langchain_pinecone import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

In [70]:
# Load Existing index 

from langchain_pinecone import Pinecone

docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
# This step reads embeddings already stored inside Pinecone,
# without needing your PDF or text_chunks.


In [71]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import Pinecone as PineconeVectorStore

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



In [85]:
docs = docsearch.similarity_search("What is this document about?", k=3)
docs


[Document(id='67b2db5d-4c73-4718-b9db-892a3d19d7a0', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 2.0, 'page_label': '3', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='er will be corrected in future editions.\nThis book is printed on recycled paper that meets Environmental Pro-\ntection Agency standards.\nThe paper used in this publication meets the minimum requirements of\nAmerican National Standard for Information Sciences-Permanence\nPaper for Printed Library Materials, ANSI Z39.48-1984.\nThis publication is a creative work fully protected by all applicable\ncopyright laws, as well as by misappropriation, trade secret, unfair com-'),
 Document(id='2233e875-f237-4222-ae56-8db363b149cc', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': 

In [73]:
# This is optional to chequek That everything is work fine it means docsearch fetching the data from the knowledge base pinecone
query = "What is this document about?"
results = docsearch.similarity_search(query, k=3)
results



[Document(id='67b2db5d-4c73-4718-b9db-892a3d19d7a0', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 2.0, 'page_label': '3', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='er will be corrected in future editions.\nThis book is printed on recycled paper that meets Environmental Pro-\ntection Agency standards.\nThe paper used in this publication meets the minimum requirements of\nAmerican National Standard for Information Sciences-Permanence\nPaper for Printed Library Materials, ANSI Z39.48-1984.\nThis publication is a creative work fully protected by all applicable\ncopyright laws, as well as by misappropriation, trade secret, unfair com-'),
 Document(id='2233e875-f237-4222-ae56-8db363b149cc', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': 

In [74]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [75]:
retrieved_docs = retriever.invoke("What is Acne?")

In [76]:
retrieved_docs

[Document(id='487cac06-edcd-49ed-a61f-20595919b0f4', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 425.0, 'page_label': '426', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='Corticosteriod —A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(id='8434ce96-3312-4460-9e83-4acaee51d5e6', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator

In [77]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.4,
    max_tokens=500,
    api_key=os.getenv("GROQ_API_KEY")
)


In [78]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [84]:
# 4. Build the final LLM chain (same job as StuffDocumentsChain in old LangChain)
# This takes the prompt → runs it through the LLM → converts output to clean text
parser = StrOutputParser()
chain = prompt | llm | parser

# 5. Full RAG pipeline (same job as RetrievalChain in old LangChain)
# Step 1: Retrieve top 3 most relevant chunks from Pinecone based on user question
# Step 2: Merge those chunks into one single context block
# Step 3: Send {context + question} to the LLM and return its answer
def rag_pipeline(query):
    docs = docsearch.similarity_search(query, k=3)
    context = "\n\n".join([d.page_content for d in docs])
    return chain.invoke({"context": context, "input": query})

# 6. Test the chatbot by asking a question
rag_pipeline(" What is caffeine")



'Caffeine is a drug that stimulates the central nervous system. It makes people more alert, less drowsy, and improves coordination. Caffeine is found naturally in coffee, tea, chocolate, and some soft drinks.'

In [1]:
# -------------------------------------------------------------
# GROQ LLM SANITY CHECK (WHY THIS BLOCK IS IMPORTANT)
# -------------------------------------------------------------
# Before using Groq inside your RAG pipeline, we MUST test:
#   1. API key loading from .env file
#   2. Authentication with the Groq server
#   3. Whether the selected model exists and is available
#   4. Whether the client can generate a basic response
#
# If this block works → your whole chatbot will work.
# -------------------------------------------------------------

from groq import Groq
import os
from dotenv import load_dotenv

# Load environment variables (GROQ_API_KEY)
load_dotenv()

# Initialize Groq client using the API key from .env
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# -------------------------------------------------------------
# Send a simple test message to verify:
#   ✔ Correct API key
#   ✔ Correct model name (llama-3.1-8b-instant)
#   ✔ Groq servers responding properly
#   ✔ Client library works
# -------------------------------------------------------------
chat = client.chat.completions.create(
    model="llama-3.1-8b-instant",         # Groq's fast + free model
    messages=[{"role": "user", "content": "Hello"}]
)

# Print the model's response to confirm everything is working
print(chat.choices[0].message.content)

# -------------------------------------------------------------
# If you see a response (like "How can I assist you today?"):
#   ✔ Your Groq LLM is fully functional.
#   ✔ Safe to integrate with RAG pipeline.
# -------------------------------------------------------------







Hello. How can I assist you today?
